In [ ]:
import sys
sys.path.append('/home/haotian/molecules_confs/Protein_test/SurfBP')
from dataset.masif.generate_prot_ply import compute_inp_surface
import os.path as osp
from glob import glob
import multiprocessing
import numpy as np
import os 
from tqdm import tqdm
import copy
from rdkit import Chem
def set_mol_positions(mol, pos):
    mol = copy.deepcopy(mol)
    for i in range(pos.shape[0]):
        mol.GetConformer(0).SetAtomPosition(i, pos[i].tolist())
    return mol 

def read_sdf(sdf_file, sanitize=False):
    supp = Chem.SDMolSupplier(sdf_file, sanitize=sanitize)
    mols_list = [i for i in supp]
    return mols_list

def write_sdf(mol_list,file):
    writer = Chem.SDWriter(file)
    for i in mol_list:
        writer.write(i)
    writer.close()


In [ ]:
all = len(glob('/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/all_pdb/*.pdb'))
generated_num = len(glob('/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/surf_all/*'))
print('Progress',generated_num*100 / all, '%', '|| Generated',generated_num)

In [ ]:
pdbfiles[0].split('/')[-1][:-8]

In [ ]:
mols = read_sdf('/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/MOAD_test_mol.sdf')

In [ ]:
pdb_dir = '/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/test_pdb'
for mol in tqdm(mols):
    mol_name = mol.GetProp('_Name')
    sdf_path = osp.join(pdb_dir, mol_name+'.sdf')
    write_sdf([mol],sdf_path)

In [ ]:
all_pdb = glob('/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/all_pdb/*.pdb')

In [ ]:
for pdbfile in all_pdb:
    if mols[0].GetProp('_Name') +'_15A.pdb' in pdbfile:
        print('yes')

In [ ]:
pdb_dir = '/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/all_pdb'
pdbfile = osp.join(pdb_dir, mols[0].GetProp('_Name') +'_15A.pdb')
osp.exists(pdbfile)

In [ ]:
out_dir = '/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/surf_test'
pdb_dir = '/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/test_pdb'
pdbfiles = np.sort(glob(pdb_dir+'/*.pdb'))
fail_list = []

def gen_ply_file(i):
    pkt_file = pdbfiles[i]
    out_name = pkt_file.split('/')[-1][:-8] #'1a4k_0'
    sdf_file = osp.join(pdb_dir, out_name + '.sdf')
    ply_file = "%s/%s_pocket_%s_res_%s.ply"%(out_dir, out_name, 8.0, 1.5)
    if osp.isfile(ply_file):
        return 
    try:
        compute_inp_surface(pkt_file, sdf_file, out_dir, out_name=out_name, mesh_res=1.5,dist_threshold=8.0)
        return
    except: 
        print('Generate featured surface failed!{}'.format(i))
        fail_list.append(i)
        return i


In [ ]:
len(glob('/home/haotian/molecules_confs/Protein_test/SurfGen/data/linker/protein/test_pdb/*'))

In [ ]:
from tqdm import tqdm
fail_list = []
pool = multiprocessing.Pool(5)
for result in tqdm(pool.imap(gen_ply_file, range(len(pdbfiles))), total=len(pdbfiles)):
    if result is not None:
        fail_list.append(result)
pool.close()
pool.join()